#### Importes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import pickle
from sklearn.feature_selection import f_classif, SelectKBest

### Comprensión empresarial

Los depósitos a largo plazo permiten a los bancos retener dinero durante un período de tiempo específico, lo que permite al banco utilizar ese dinero para mejorar sus inversiones. Las campañas de marketing de este producto se basan en llamadas telefónicas. Si un usuario no se encuentra disponible en un momento dado, entonces se le volverá a llamar de nuevo en otro momento.

Descripción del problema

El banco portugués está teniendo una disminución en sus ingresos, por lo que quieren poder identificar a los clientes existentes que tienen una mayor probabilidad de contratar un depósito a largo plazo. Esto permitirá que el banco centre sus esfuerzos de marketing en esos clientes y evitará perder dinero y tiempo en clientes que probablemente no se suscribirán.

Para abordar este problema crearemos un algoritmo de clasificación que ayude a predecir si un cliente contratará o no un depósito a largo plazo.

- age. Edad del cliente (numérico)

- job. Tipo de trabajo (categórico)

- marital. Estado civil (categórico)

- education. Nivel de educación (categórico) 

- default. ¿Tiene crédito actualmente? (categórico) / FUERA

- housing. ¿Tiene un préstamo de vivienda? (categórico)

- loan. ¿Tiene un préstamo personal? (categórico)

- contact. Tipo de comunicación de contacto (categórico) /FUERA

- month. Último mes en el que se le ha contactado (categórico) 

- day_of_week. Último día en el que se le ha contactado (categórico)

- duration. Duración del contacto previo en segundos (numérico) / FUERA

- campaign. Número de contactos realizados durante esta campaña al cliente (numérico) / FUERA

- pdays. Número de días que transcurrieron desde la última campaña hasta que fue contactado (numérico) / FUERA

- previous. Número de contactos realizados durante la campaña anterior al cliente (numérico) /FUERA

- poutcome. Resultado de la campaña de marketing anterior (categórico) / FUERA

- emp.var.rate. Tasa de variación del empleo. Indicador trimestral (numérico)

- cons.price.idx. Índice de precios al consumidor. Indicador mensual (numérico)

- cons.conf.idx. Índice de confianza del consumidor. Indicador mensual (numérico) 

- euribor3m. Tasa EURIBOR 3 meses. Indicador diario (numérico) 

- nr.employed. Número de empleados. Indicador trimestral (numérico) /FUERA 'Necesito datos de los clientes, no de los chambeadores'

- y. TARGET. El cliente contrata un depósito a largo plazo o no (categórico) 


## PASO 1 : Planteamos nuestro problema o nuestro target a investigar


#### ¿Que cliente contratará o no un depósito a largo plazo?

## PASO 2: Recopilacion de datos

Recopilamos la informacion de nuestro DataSet

In [ ]:
df = pd.read_csv("/workspaces/machine-learning-elius123ef/data/raw/bank-marketing-campaign-data.csv", sep=";")

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Paso 3: Análisis Descriptivo

In [3]:
df.shape

(41188, 21)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

## Paso 4: Limpieza de Datos

### Normalizar valores de texto

In [23]:

for col in ['job','marital','education','default','housing','loan','y']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()

### Mapear target


In [20]:
df['y_bin'] = df['y'].map({'yes': 1, 'no': 0})
df['y'].value_counts(dropna=False)

y
no     36537
yes     4639
Name: count, dtype: int64

In [21]:
df['y_bin'].value_counts(dropna=False)

y_bin
0    36537
1     4639
Name: count, dtype: int64

### Buscamos valores duplicados:

In [9]:
df.duplicated().sum()

np.int64(0)

##### Observaciones:

No encontramos valores duplicados.

### Buscamos valores nulos o sin conocer 

#### Nulos por columna


In [ ]:
df.isna().sum().sort_values(ascending=False)

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
y_bin             0
dtype: int64

#### Proporción de 'unknown' en categóricas


In [35]:
cat_cols = ['job','marital','education','default','housing','loan']
for col in cat_cols:
    if col in df.columns:
        unk_rate = (df[col] == 'unknown').mean()
f"{col}: cantidad de veces que aparece unknown = {unk_rate:.3f}%"

'loan: cantidad de veces que aparece unknown = 0.024%'

### Análisis de balance de clases

- Objetivo: Ver si el dataset está desbalanceado

Obtenemos la Tasa de positivos (yes). 

In [40]:
pos_rate = float(df['y_bin'].mean())
pos_rate

0.11266271614532737

### Limpieza de datos: Eliminar información irrelevante



#### Buscamos que informacion es irrelevante para nosotros y asi proceder a eliminarla

In [10]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

### Eliminacion de columnas innecesarias:

**NOTA: Nos aseguramos de hacer una copia para asi no perder el df original.**

In [11]:
df_copy = df.copy()
df_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [12]:
df_copy.drop(["contact", "month", "day_of_week", "campaign", "pdays", "poutcome",
            "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"], axis=1, inplace=True)
df_copy

,age,job,marital,education,default,housing,loan,duration,previous,y
0,56,housemaid,married,basic.4y,no,no,no,261,0,no
1,57,services,married,high.school,unknown,no,no,149,0,no
2,37,services,married,high.school,no,yes,no,226,0,no
3,40,admin.,married,basic.6y,no,no,no,151,0,no
4,56,services,married,high.school,no,no,yes,307,0,no
...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,334,0,yes
41184,46,blue-collar,married,professional.course,no,no,no,383,0,no
41185,56,retired,married,university.degree,no,yes,no,189,0,no
41186,44,technician,married,professional.course,no,no,no,442,0,yes


#### Observaciones

- Eliminamos las columnas `contact` `month` `day_of_week` `campaign` `pdays` `poutcome` `emp.var.rate` `cons.price.idx` `cons.conf.idx` `euribor3m` `nr.employed` , ya que considero que son variables que no influirian en la decision de algun cliente, y que influye mas en las relaciones entre los bancos.

## Paso 5: Análisis de Variables

### Análisis de Variables Univariante

### Análisis de Variables Univariante Categóricas

In [13]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41176 entries, 0 to 41187
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        41176 non-null  int64 
 1   job        41176 non-null  object
 2   marital    41176 non-null  object
 3   education  41176 non-null  object
 4   default    41176 non-null  object
 5   housing    41176 non-null  object
 6   loan       41176 non-null  object
 7   duration   41176 non-null  int64 
 8   previous   41176 non-null  int64 
 9   y          41176 non-null  object
dtypes: int64(3), object(7)
memory usage: 3.5+ MB


## Paso 6: Ingeniería de características

## Paso 7: Split (dos métodos o enfoques)

## Paso 8: Scaling & Encoding

## Paso 9: Selección de características